### Introdução à Engenharia Biomédica (P4 - 2021/2022)
# Aula Prática #5 - Extração de Features de Biosinais

# <span style='color:#484848;'> Biosignal Feature Extraction </span>

### <span style='color:#00aba1;'> Keywords </span>

`ECG`, `BioSSPy`, `Feature Extraction`, `HRV`

### <span style='color:#00aba1;'> Notebook Info </span>

**Contributor(s):** Rafael Silva, Hugo Plácido da Silva and Ana Fred

**Date of creation:** 13/06/2022

**Last update:** 13/06/2022

# I. Overview
<br>
<div class="title"style="width:100%; background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

# <div style="color:#fbb144"> 1. Introduction </div>

Time and frequency representations of signals often contain valuable information that can be more easily extracted and analyzed. Such information can be statistical measures (e.g., mean, maximum and minimum amplitudes, variance), and be related to the nature and morphology of the signal (e.g., location of P, QRS, and T waves, series of RR intervals – for ECG).

Regarding biosignals, this is an extremely important task when developing automatic methods for physiological monitoring and medical diagnosis. In this class you will be presented with some biosignal processing and feature extraction tools for biomedical research.

<img src="_Resources/gram-1024x723.png" style="width:70%">
<a href="https://usq.pressbooks.pub/anatomy/chapter/6-2-cardiac-muscle-and-electrical-activity/">Reference</a>

# <div style="color:#fbb144"> 2. Objectives</div>

* Understand the objective of feature extraction in the context of biosignals

* Get acquainted with the main BioSPPy capabilities for biosignals

# II. BioSPPy for ECG Signals
<br>
<div class="title"style="width:100%; background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

<img src="_Resources/biosppy_logo.png"/>

Python is often referred to as "the batteries included language", meaning that there are plenty of useful modules either bundled or contributed by the community. Biosignals are no exception and some libraries already bundle basic signal processing and analytics components.

The ECG module available in BioSPPy provides methods for ECG processing (visualization, filtering, segmentation, etc.).


Make sure you have BioSSPy installed. If not, run the following command in a code cell:

```python
!pip install biosppy
```

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import biosppy as bp

Let's load an ECG file and use the function `signals.ecg.ecg()`, that calls several functions that automatically filter the signal, find R peaks, compute the instantaneous heart rate and segments the ECG waveforms. This function also provides an interactive interface to visualize its outputs.

In [ ]:
# Load file
fs = 1000. # Hz
ecg_op3 = np.loadtxt('_Resources/ecg_op3.csv')

# To use external viewer
%matplotlib

# BioSPPy
bp.signals.ecg.ecg(ecg_op3, fs, show=True);

<img src="_Resources/biosppy_ecg.png" style="width:80%"/>

Let's explore how this function works:

```python
def ecg(signal=None, sampling_rate=1000., path=None, show=True):
    """Process a raw ECG signal and extract relevant signal features using
    default parameters.

    Parameters
    ----------
    signal : array
        Raw ECG signal.
    sampling_rate : int, float, optional
        Sampling frequency (Hz).
    path : str, optional
        If provided, the plot will be saved to the specified file.
    show : bool, optional
        If True, show a summary plot.

    Returns
    -------
    ts : array
        Signal time axis reference (seconds).
    filtered : array
        Filtered ECG signal.
    rpeaks : array
        R-peak location indices.
    templates_ts : array
        Templates time axis reference (seconds).
    templates : array
        Extracted heartbeat templates.
    heart_rate_ts : array
        Heart rate time axis reference (seconds).
    heart_rate : array
        Instantaneous heart rate (bpm).

    """

    # check inputs
    if signal is None:
        raise TypeError("Please specify an input signal.")

    # ensure numpy
    signal = np.array(signal)

    sampling_rate = float(sampling_rate)

    # filter signal
    order = int(0.3 * sampling_rate)
    filtered, _, _ = st.filter_signal(signal=signal,
                                      ftype='FIR',
                                      band='bandpass',
                                      order=order,
                                      frequency=[3, 45],
                                      sampling_rate=sampling_rate)

    # segment
    rpeaks, = hamilton_segmenter(signal=filtered, sampling_rate=sampling_rate)

    # correct R-peak locations
    rpeaks, = correct_rpeaks(signal=filtered,
                             rpeaks=rpeaks,
                             sampling_rate=sampling_rate,
                             tol=0.05)

    # extract templates
    templates, rpeaks = extract_heartbeats(signal=filtered,
                                           rpeaks=rpeaks,
                                           sampling_rate=sampling_rate,
                                           before=0.2,
                                           after=0.4)

    # compute heart rate
    hr_idx, hr = st.get_heart_rate(beats=rpeaks,
                                   sampling_rate=sampling_rate,
                                   smooth=True,
                                   size=3)

    # get time vectors
    length = len(signal)
    T = (length - 1) / sampling_rate
    ts = np.linspace(0, T, length, endpoint=True)
    ts_hr = ts[hr_idx]
    ts_tmpl = np.linspace(-0.2, 0.4, templates.shape[1], endpoint=False)

    # plot
    if show:
        plotting.plot_ecg(ts=ts,
                          raw=signal,
                          filtered=filtered,
                          rpeaks=rpeaks,
                          templates_ts=ts_tmpl,
                          templates=templates,
                          heart_rate_ts=ts_hr,
                          heart_rate=hr,
                          path=path,
                          show=True)

    # output
    args = (ts, filtered, rpeaks, ts_tmpl, templates, ts_hr, hr)
    names = ('ts', 'filtered', 'rpeaks', 'templates_ts', 'templates',
             'heart_rate_ts', 'heart_rate')

    return utils.ReturnTuple(args, names)
```

We can retrieve all outputs using a single output variable:

In [ ]:
# BioSPPy
ecg_data = np.loadtxt('_Resources/ecg_op3.csv')
out = bp.signals.ecg.ecg(ecg_data, fs, show=False)
out.keys()

This way we can perform our own operations:

In [ ]:
# Extract data
t = out['ts']
ecg_filtered = out['filtered']
rpeaks = out['rpeaks']

# Plot
%matplotlib inline

plt.plot(t, ecg_filtered, color='tab:blue', linewidth=1, label='ECG signal')
plt.plot(t[rpeaks], ecg_filtered[rpeaks],'kx', label='R-peaks')
plt.xlim([0,15])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend(loc='upper right')
plt.show()

In [ ]:
out['ts']

In [ ]:
rpeaks

In [ ]:
t[rpeaks]

An alternative is to directly use the functions of the `ecg` module, that also contains other methods for signal segmentation, feature extraction and signal quality assessment.

In [ ]:
dir(bp.signals.ecg)

#### EXERCISE

1. Load the `ecg_op2.csv` ($fs=1000 Hz$) into the variable `ecg_op2` (7th column) and then, using the `ecg` BioSSPy module, run the `ecg()` function and save the output into the variable `ecg_out`.

2. Using matplotlib's functionalities, use the `templates_ts` and `templates` outputs to replicate the `ecg` template window. Note that you should transpose both arrays. Use the following plot style:
* color = (your choice)
* linewidth = 1
* alpha = 0.08 (transparency)

*Hint: you can use the `.T` method to transpose NumPy arrays.*

3. Aditionally, use the `np.mean()` function to add the mean template waveform to the previous plot using:
* color = 'black'
* linewidth = 1
* linestyle = '--'
* label = 'Mean'

You should provide a legend for the mean waveform.

*Hint: when using the np.mean function, use the axis parameter to compute the mean of the array columns.*

# III. Heart Rate Variability (HRV)
<br>
<div class="title"style="width:100%; background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

Besides directly analysing ECG, the heart rate variability (HRV) is also very common and useful when dealing with heart-based signals. This signal allows to understand variations to the frequency of the ECG, which is related to alterations in the overall state of a person.

Both HR and HRV are modulated by the autonomic nervous system, which includes sympathetic and parasympathetic nerve branches. These branches innervate parts of the atria of the heart. Sympathetic stimulation increases HR and parasympathetic stimulation decreases it. In addition to the natural variations in heart rate caused by breathing, HRV appears to be sensitive to factors such as mental stress, sleep and fatigue, chronic illness, and others. Elevated HRV is also related to well-being and fitness.

HRV may be evaluated by a variety of methods. We will describe some of them below.

#### DISCUSSION:
Besides the ECG, what other signal could be used to perform HRV analysis?

# <div style="color:#fbb144"> 1. RR-Intervals (RRIs) </div>

An RR-Interval (RRI) corresponds to the time difference between two consecutive R-peaks of the ECG.

<img src="./_Resources/RR-interval.png" style="width:30%"/>

<br>
<br>

$$ RR_{i+1} = R_{i+1} - R_{i},\ \ i = 0, \ldots ,N-1 $$

In [ ]:
# Extract R-peaks
rpeaks = out['rpeaks']/fs*1000 # convert sample indexes to ms

# Compute RR Intervals
rri = []
for i in range(len(rpeaks)-1):
    rri.append(rpeaks[i + 1] - rpeaks[i])

rri = np.array(rri) # convert to numpy array

rri

In [ ]:
# Plot RRIs
plt.figure(figsize=(10,3))
plt.plot(rri, lw=1, color='k', marker='v', markerfacecolor='orange')
plt.ylabel('RR-intervals (ms)')
plt.show()

With the RRI series, we can compute statistical metrics such as mean and standard deviation:

In [ ]:
# Compute mean and sd
rr_mean = np.mean(rri)
rr_sd = np.std(rri)

print(f'Mean = {rr_mean}, SD = {rr_sd}')

We can also represent the data with a histogram:

In [ ]:
# Plot histogram
bins = np.arange(np.min(rri), np.max(rri)+10, 10)

plt.hist(rri, bins, facecolor='skyblue', edgecolor='black')
plt.title('RR Distribution')
plt.xlabel('RR Interval (ms)')
plt.ylabel('Count')
plt.show()

# <div style="color:#fbb144"> 2. Heart Rate (HR) </div>

Heart Rate (HR) corresponds to the number of R-peaks that occur in a minute. Alternatively, we can compute the instantaneous Heart Rate:


$$ {HR}_{i+1} = \frac{60}{R_{i+1} - R_{i}} (bpm),\ \ i = 0, \ldots ,N-1 $$

In [ ]:
# Compute heart rate
ts = out['rpeaks'][1:]/fs
hr = fs * (60.0 / np.diff(out['rpeaks']))

# Plot
plt.figure(figsize=(10,3))
plt.plot(ts, hr, lw=1, color='k', marker='v', markerfacecolor='orange')
plt.ylabel('Heart Rate (bpm)')
plt.xlabel('Time (s)')
plt.show()

# <div style="color:#fbb144"> 3. Root Mean Square of Successive Differences (RMSSD) </div>

Another important feature for the analysis of HRV is the root mean square of SDs between adjacent RR-intervals (RMSSD), which reflects parasympathetic activity.

<img src="_Resources/rmssd-walkthrough.png" style="width:40%">
<a href="https://imotions.com/blog/heart-rate-variability/">Reference</a>

$$
R M S S D=\sqrt{\frac{1}{N-1} \sum_{i=1}^{N-1}\left(R R_{i+1}-R R_{i}\right)^{2}}
$$

In [ ]:
rpeaks = out['rpeaks']/fs*1000 # ms
rr_int = np.diff(rpeaks)
rr_diff = np.diff(rr_int)

rmssd = np.sqrt(np.mean(rr_diff**2))
print(f'RMSSD = {rmssd} ms')

# <div style="color:#fbb144"> 4. Poincaré plot </div>

Another feature to examine is the Pointcaré plot, which is generated as the plot:

$$RR_{i+1} \textrm{vs.} RR_i$$


In this plot, the standard descriptor 1 (SD1) describes the short-term variability in the R-R intervals, whereas the standard descriptor 2 (SD2) describes the long-term variability (29). SD1 mostly represents parasympathetic heart influence, whereas SD2 reflects sympathetic and parasympathetic inputs to the heart.

<img src="_Resources/poincare.png" style="width:30%"/>

In [ ]:
import matplotlib as mpl
x = rri[:-1]
y = rri[1:]
rr_mean = np.mean(rri)

# Plot
fig, ax = plt.subplots(figsize=(6,6))
ax.set_title('Poincaré Plot', fontsize=14)
ax.set_xlabel('$RR_i$ (ms)', fontsize=12)
ax.set_ylabel('$RR_{i+1}$ (ms)', fontsize=12)

# Plot Poincaré data points
ax.scatter(x, y, marker='.', color='blue', alpha=0.5,s=100)
ax.set_xlim([np.min(rr_int) - 50, np.max(rr_int) + 50])
ax.set_ylim([np.min(rr_int) - 50, np.max(rr_int) + 50])
ax.set_aspect(1./ax.get_data_ratio())


# Draw RRi+1=RRi
lims = [np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()])]  # max of both axes

ax.plot(lims, lims, color='grey', linestyle='--')


# Draw ellipse
sd1 = np.std(np.subtract(x, y) / np.sqrt(2))
sd2 = np.std(np.add(x, y) / np.sqrt(2))
area = np.pi * sd1 * sd2


ellipse_ = mpl.patches.Ellipse((rr_mean, rr_mean), sd1 * 2, sd2 * 2, angle=-45, edgecolor='black', facecolor="None")
ax.add_artist(ellipse_)

ax.grid()

print(f'SD1 = {sd1} \nSD2 = {sd2} \nArea = {area} \nSD1/SD2 = {sd1/sd2}')



# <div style="color:#fbb144"> 5. Frequency Analysis </div>

Furthermore, frequency-domain analysis describes the high and low frequency (LV) rates of variability changes, which correlate to the activity of distinct ANS branches. The unique contributions of the parasympathetic and sympathetic systems were discovered by using these frequency range discrepancies in HRV studies.

<img src="_Resources/FFT-spectrum.png"/>

#### EXERCISE:

1. Use BioSPPy's capabilities to perform a spectral analysis of the `ecg_op2` signal. 

*Hint: Use the function `signals.tools.welch_spectrum()`.*

2. Use Matplotlib to plot the ECG spectrum.

*Hint: check the previous notebook.*

# IV. Atrial Fibrillation (AFib)
<br>
<div class="title"style="width:100%; background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

Atrial Fibrillation (AFib) is a type of heart arrhythmia, an uncoordinated activity of the heart chambers, in which the heart presents an irregular and high-rate electrical activity of the atria. It is the most common type of heart arrhythmia in the Western World with an estimated prevalence of 46.3 million people around the globe, and it is one of the main causes of stroke since it increases the risk of having one by 4 to 5 fold.

When observing the ECG, AFib is characterized by having irregular heart rhythms, absence of P-waves, nonexistence of an isoelectric baseline and variable ventricular rate. Also, P-wave activity can be replaced by fibrillatory waves (f-waves).

<img src="_Resources/AFib.png" style="width:70%"/>

Since it drastically changes heart rhythm, AFib can be identified using HRV metrics:

In [ ]:
from functions import *

ecg_af = np.loadtxt('_Resources/ecg_af.csv')

plot_ecg(ecg_af, fs)
plot_templates(ecg_af, fs)
compute_rri(ecg_af, fs)
compute_hr(ecg_af, fs)
compute_rmssd(ecg_af, fs)
plot_poincare(ecg_af, fs)

# Further Reading
<br>
<div class="title"style="width:100%; background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

* Dong, J. (2016). *The role of heart rate variability in sports physiology (Review)*. Experimental and Therapeutic Medicine, 11, 1531-1536. https://doi.org/10.3892/etm.2016.3104
* Makivic , B., &amp; Bauer, P. *Heart Rate Variability Analysis in Sport*. Aspetar Sports Medicine Journal - Heart rate variability analysis in sport. Retrieved June 6, 2022, from https://www.aspetar.com/journal/viewarticle.aspx?id=248#.Yp4f2qjMJPZ 
* Valencell. *Heart Rate Variability Review*. Retrieved June 6, 2022, from https://valencell.com/blog/heart-rate-variability-review/

<tr>
<td> <img src="_Resources/IT.png" alt="Drawing" style="width:200px"/> </td>

<td> <img src="_Resources/IST.png" alt="Drawing"
style="width:200px"/> </td>
</tr>